In [27]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import imodelsx.process_results
import sys
import datasets
import numpy as np
from copy import deepcopy
import clin.eval
import clin.modules.prune
import clin.modules.evidence
import clin.modules.omission
import clin.modules.status
import clin.llm
import clin.parse
from collections import defaultdict
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'

sys.path.append('../experiments/')
results_dir = '../results/'

r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)

# get data for eval
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
df_val = pd.DataFrame.from_dict(dset['validation'])
df = pd.DataFrame.from_dict(dset['test'])
# df = pd.concat([val, test])
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
dfe = df.iloc[nums]
n = len(dfe)

# single row to investigate
row = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')].iloc[0]
sv = clin.modules.status.StatusVerifier()

# (
#     r.groupby(['checkpoint', 'n_shots'])[['f1', 'recall', 'precision']].mean()
#     .style.format(precision=3).background_gradient(cmap='Blues')
# )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 1/1 [00:00<00:00, 1015.82it/s]


  0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
# compare values for a single row
row_df = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')]
rc = row_df[[c for c in row_df.columns if '___' in c]]
# create multindex columns by splitting on '___'
rc = rc.rename(columns={c: tuple(c.split('___')) for c in rc.columns})

# convert tuple column names to multiindex
rc.columns = pd.MultiIndex.from_tuples(rc.columns)
rc = rc.T.reset_index()
rc = rc.rename(columns={
    'level_0': 'metric',
    'level_1': 'verification',
}).pivot_table(index='verification', columns='metric', values=0).round(3)
rc

metric,f1,precision,recall
verification,,,
ev,0.918,0.918,0.918
original,0.919,0.918,0.921
ov,0.906,0.870,0.944
ov_pv,0.936,0.946,0.926
ov_pv_ev,0.938,0.949,0.926
pv,0.926,0.948,0.906


In [29]:
# print errs
# mets = clin.eval.calculate_metrics(row['dict_ov_pv_ev'], dfe, verbose=True)

# Test on medication status

In [30]:
settings = [k for k in row.keys() if k.startswith('dict_') and not k.startswith('dict_evidence')]
med_status_dicts_list = [row[k] for k in settings]

In [32]:
# add medication status eval
# med_status_dicts_list = [[clin.parse.parse_response_medication_list(r.iloc[i]['resps'][j]) for j in range(n)] for i in range(len(r))]
d = defaultdict(list)
accs_cond, f1s_macro_cond = clin.eval.eval_medication_status(med_status_dicts_list, dfe)
d['acc_cond'] = accs_cond
d['f1_macro_cond'] = f1s_macro_cond
d['index'] = settings
print(d)
(
    pd.DataFrame.from_dict(d).set_index('index') #[['acc_cond', 'f1_macro_cond']]
    .style.format(precision=3).background_gradient(cmap='Blues')
)

100%|██████████| 7/7 [00:00<00:00, 529.11it/s]

defaultdict(<class 'list'>, {'acc_cond': [0.8878205128205128, 0.8910256410256411, 0.8878205128205128, 0.8878205128205128, 0.8910256410256411, 0.8782051282051282, 0.8782051282051282], 'f1_macro_cond': [0.6603305629205868, 0.6625776019715414, 0.6603305629205868, 0.6603305629205868, 0.6625776019715414, 0.6059983502202301, 0.6059983502202301], 'index': ['dict_original', 'dict_ov', 'dict_pv', 'dict_ev', 'dict_ov_pv', 'dict_ov_pv_ev', 'dict_sv']})


,acc_cond,f1_macro_cond
index,,
dict_original,0.888,0.660
dict_ov,0.891,0.663
dict_pv,0.888,0.660
dict_ev,0.888,0.660
dict_ov_pv,0.891,0.663
dict_ov_pv_ev,0.878,0.606
dict_sv,0.878,0.606


In [36]:
dfe.iloc[0]

[autoreload of clin.modules.status failed: Traceback (most recent call last):
  File "/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/chansingh/clinical-extract-and-check/clin/modules/status.py", line 12, in <module>
    _%#NAME#%_ tolerated his chemotherapy well with minimal nausea and no emesis. At the time of discharge, he was in no apparent distress and was afebrile

index                                                                       6
snippet                     I will recommend discontinuing the alcohol wit...
active_medications                        ["Lexapro", "Ativan", "Inderal LA"]
discontinued_medications                                                   []
neither_medications                                                ["Celexa"]
Name: 1, dtype: object